# Federated collaborative filtering 

Your task is to implement federated collaborative filterng model for privacy-preserving personalized recommendation system.  
In the Federated Learning paradigm, a master machine learning model is distributed to user clients, the clients use their locally stored data and model for both inference and calculating model updates. The model updates are sent back and aggregated on the server to update the master
model then redistributed to the clients. In this paradigm, the user data never leaves the client, greatly enhancing the user’ privacy, in contrast to the traditional paradigm of collecting, storing and processing user data on a backend server beyond the user’s control. The federated updates to the model are based on a gradient approach. 

Specifically, the method defines three core components as below.
1. All the item factor vectors $y_i, i = 1, ... , M$ are updated on the server and then distributed to each client $u$.
2. The user factor vectors $x_u, u = 1, ... , N$ are updated locally on the client $u$, using the user $u$’s own data and the $y_i, i = 1, ... , M$ from the server.
3. The updates through the gradients $\delta y_{ui}$ are calculated for the item $i$ on each client $u$ and transmitted to the server where the gradients are aggregated to update $y_i$.

The cost function optimizing across all users $u$ and the items $i$ over the confidence levels $c_{ui}$ is then given as

$$J = \sum_u \sum_i c_{ui}(p_{ui} - x_u^T y_i)^2 + \lambda (\sum_u||x_u||^2 + \sum_i ||y_i||^2),$$
with a regularization parameter $\lambda$,
where 
$$p_{ui} = \begin{cases}  
1 & r_{ui} > 0,\\
0 & r_{ui} = 0
\end{cases}$$
 $$c_{ui} = 1 + \alpha r_{ui}$$

User factors are updated by directly solving an equation

$$ \frac{\partial J(x_u)}{\partial x_u} = 0 $$
Thus, we obtain the solution
$$ x_u = (YC^uY^T + \lambda I)^{-1}YC^up(u),$$
where $ C^u \in \mathbb{R}^{N \times N}$ is a diagonal matrix with $C_{ii}^u = c_{ui}$ and $p(u) \in \mathbb{R}^{N \times 1}$ contains the $p_{ui}$ values for the user $u$.

Item factors are updated on the master server as

$$ y_i = y_i - \gamma \frac{\partial J}{\partial y_i},$$

where gamma is gain parameter.

$$ \frac{\partial J}{\partial y_i} = -2 \sum_u f(u,i) +2 \lambda y_i, $$
where 
$$ f(u,i) = \left[ c_{ui}(p_{ui} - x_u^T y_i)\right]x_u.$$

To familiarise yourself with the method, read the paper https://arxiv.org/pdf/1901.09888.pdf


In [ ]:
#!pip install ipypb

In [ ]:
#!pip install --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

In [1]:
from scipy.sparse import csr_matrix
from ipypb import irange

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from polara import get_movielens_data
from polara.evaluation.pipelines import random_grid, find_optimal_config

## Download dataset

For this task we will use popular MovieLens dataset.

In [3]:
data = get_movielens_data(
    get_genres=False, # we won't need genres
    split_genres=False,
    include_time=True
)

data.head()

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# 1. Preprocess data  (5 pts)
Do the preprocessing, including data split and index transformation. You are recommended to use functions from the seminar on evaluation.

## Data split 


Split data to train/validation/test datasets, using leave-last-out scheme for holdout sampling.

In [4]:
def leave_last_out(data, userid='userid', timeid='timestamp'):
    data_sorted = data.sort_values('timestamp')
    holdout = data_sorted.drop_duplicates(
        subset=['userid'], keep='last'
    ) # split the last item from each user's history
    remaining = data.drop(holdout.index) # store the remaining data - will be our training
    return remaining, holdout

In [5]:
training_, holdout_ = leave_last_out(data)

In [6]:
training_, holdout_val_ = leave_last_out(training_)

## Transform index

Transform index in train dataset and reindex validation and test datasets.

In [7]:
def transform_indices(data, users, items):
    data_index = {}
    for entity, field in zip(['users', 'items'], [users, items]):
        new_index, data_index[entity] = to_numeric_id(data, field)
        data = data.assign(**{f'{field}': new_index}) # makes a copy of dataset!
    return data, data_index

def to_numeric_id(data, field):
    idx_data = data[field].astype("category")
    idx = idx_data.cat.codes
    idx_map = idx_data.cat.categories.rename(field)
    return idx, idx_map

def reindex_data(data, data_index, fields=None):
    if fields is None:
        fields = data_index.keys()
    if isinstance(fields, str): # handle single field provided as a string
        fields = [fields]
    for field in fields:
        entity_name = data_index[field].name
        new_index = data_index[field].get_indexer(data[entity_name])
        data = data.assign(**{f'{entity_name}': new_index}) # makes a copy of dataset!
    return data

In [8]:
train_data, data_index = transform_indices(training_, 'userid', 'movieid')

In [9]:
test_data = reindex_data(holdout_, data_index)
val_data = reindex_data(holdout_val_, data_index)

In [10]:
test_data = test_data.query('movieid >= 0')

In [11]:
# sorting data by user id for correct evaluation
test_data = test_data.sort_values('userid') 
val_data = val_data.sort_values('userid')

In [12]:
data_description = dict(
    users = data_index['users'].name, # user field
    items = data_index['items'].name, # item field
    test_users = test_data[data_index['users'].name].values,
    holdout_items = test_data[data_index['items'].name].values,
    val_users = val_data[data_index['users'].name].values,
    holdout_val_items = val_data[data_index['items'].name].values,
    n_users = train_data[data_index['users'].name].nunique(), 
    n_items = train_data[data_index['items'].name].nunique()
)

# 2. Building/training a recommender model (20 pts)

## Preparing data in matrix form

Complete the function to obtain sparse matrix from training data.

In [14]:
def matrix_from_observations(data, userid, itemid, feedback=None, shape=None, dtype=None):
    '''Return sparse matrix, obtained from training data'''
    useridx = data[userid]
    itemidx = data[itemid]
    values = data[feedback]
    return csr_matrix((values, (useridx, itemidx)))   

In [15]:
data_matrix = matrix_from_observations(train_data, userid='userid', itemid='movieid', feedback='rating')
data_matrix

<6040x3704 sparse matrix of type '<class 'numpy.int64'>'
	with 988129 stored elements in Compressed Sparse Row format>

## The model

In [16]:
n_users = data_description['n_users']
n_items = data_description['n_items']
n_factors = 50
seed = 42

In [17]:
# initialization of user and item factors
random_state = np.random.RandomState(seed)
user_factors = random_state.normal(0, 0.01, size=(n_users, n_factors))
item_factors = random_state.normal(0, 0.01, size=(n_items, n_factors))

## Server side

Complete the functions to update item factors.


In [ ]:
def update_global_model(matrix, X, Y, regularization, gain, alpha, n_iter):
    for _ in range(n_iter):
        item_factor_update(matrix, X, Y, regularization, alpha, n_iter)
    Y -= gain # update Y


def item_factor_update(matrix, X, Y, regularization, alpha, n_iter=4):
    '''Update item factor vectors'''
    n_users , _ = X.shape
    grad = 2 * regularization * Y # initialize gradient as a regularization term
    for u in range(n_users):
        grad += fetch_neg_client_grad(matrix, X, Y, u, alpha) # add calculated gradient for a user
    

def fetch_neg_client_grad(matrix, X, Y, u, alpha):
    '''
    Fetches for entire item collection at once
    f(u, i) = c_ui * (p_ui - yi^T xu) xu
    f(u, Y) = C_u *  (p_u  -   Y xu ) xu^T = - (Y xu + (Cu-1)*(Yxu - pu)) xu^T
    '''
    indptr = matrix.indices # get indptr from sparse matrix
    inds = matrix.indptr # get indices from sparse matrix
    r = matrix.data # get ratings
    
    xu = X[u] # get user's vector  
    user_neg_grad_val = ... # get negative gradient values for user 
    return # negative gradient values for user

In [23]:
data_matrix.data, data_matrix.indices, data_matrix.indptr

(array([5, 5, 4, ..., 4, 4, 5]),
 array([   0,  144,  253, ..., 3491, 3506, 3573], dtype=int32),
 array([     0,     51,    178, ..., 987669, 987790, 988129], dtype=int32))

In [40]:
data_matrix.indptr[2:3]

array([178], dtype=int32)

In [41]:
data_matrix.indices[178]

101

In [42]:
data_matrix[1, 101]

0

## Client side

Complete the functions to update user factors.

In [ ]:
def update_local_models(matrix, X, Y, regularization, alpha):
    n_users, n_factors = X.shape
    YtY = Y.T @ Y + regularization * np.eye(n_factors)
    for u in range(n_users): # to imitate locally updating on the client side
        X[u] = user_factor_update(Y, YtY, matrix, u, alpha) # each client does independently


def user_factor_update(Y, YtY, matrix, u, alpha):
    '''
    Return the solution of equation
    Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
    '''
    A, b = solve_linear_equation(Y, YtY, matrix, u, alpha)
    return # the solution of equation


def solve_linear_equation(Y, YtY, matrix, u, alpha):
    '''
    Return A = YtCuY + regularization * I and b = YtCuPu
    to use it later for solving the equation
    Xu = (YtCuY + regularization * I)^-1 (YtCuPu)

    Following identity might be helpful
    YtCuY + regularization * I = YtY + regularization * I + Yt(Cu-I)Y
    '''
    indptr = ... # get indptr from sparse matrix 
    inds = ... # get indices from sparse matrix
    r = ... # get ratings 
    
    Ynnz = ... # get rated items factors 
    A = ... # accumulate YtCuY + regularization * I in A
    b = ... # accumulate YtCuPu in b
    return A, b  

## Model Training
Complete the function to build model. Choose the hyper-parameters and store it in config. Train your model on chosen hyper-parameters.

In [ ]:
def build_model(model_config, trainset, trainset_description):
    train_matrix = ... # get sparse matrix
    random_state = np.random.RandomState(seed)
    user_factors = ... # initialize user factors
    item_factors = ... # initialize user factors

    for epoch in tqdm(range(n_epochs)):
        ... # in each epoch update global model and then update local models
            
    return user_factors, item_factors

In [ ]:
# create config with the following hyper parameters:
# seed, n_factors, regularization, gain, alpha, n_epochs, n_iter.
config = {...}

In [ ]:
params= build_model(config, train_data, data_description) 

# 3. Evaluation (5 pts)

Complete the functions to predict relevance scores. Using predcted scores generate top-n recommendations. Evaluate quality of recommendations and report HR@5 and MRR@5 etrics. Remember that we don't want to recommend items from the user's history. You are recommended to use functions from the previous seminars.

In [ ]:
topn = 5

In [ ]:
# Complete model scoring function to predict relevance scores for test user-item pairs.
def model_scoring(model_params, testset, testset_description):
    user_factors, item_factors = ... # get user and item factros
    test_user_factors = ... # get user factros for test users
    scores = ... # get scores   
    return scores

In [ ]:
# Complete model recommendations function to generate top-n recommendations using predcted scores.
def model_recom_func(model_scores, topn=5):
    ...
    return recommendations

In [ ]:
# Complete function to evaluate quality of recommendations
def evaluate_func(model_recoms, holdout, holdout_description, topn=5):
    ...
    return hr, mrr

## Evaluate model

In [ ]:
userid = data_description['users']
seen_idx_mask = train_data[userid].isin(data_description['test_users'])
testset = train_data[seen_idx_mask]

In [ ]:
test_scores = ... # get scores
recoms = ... # get recommended items
hr, mrr = ... # get metrics
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')

# Quick check

Compare your model with one or two baselines and report the results. You can use code from previous seminars. 

In [ ]:
...

# 4. Hyper-parameter selection (5 pts)

Complete the function to tune hyper-parameters of your model to provide the best quality. Train the model with the best hyper-parameters and report HR and MRR on test dataset.

In [ ]:
# you are free to change hp values
params_dict = dict(n_factors = [10, 20, 50, 100],
                  regularization = np.linspace(1, 5, 5).astype(int),
                  gain = np.logspace(-4, -2, 5),
                  alpha = np.linspace(1, 5, 5).astype(int),
                  n_epochs = [10, 15, 20, 25],
                  n_iter = np.linspace(4, 20, 5).astype(int))

In [ ]:
param_grid, param_names = random_grid(params_dict, n=30)

In [ ]:
def hp_selection(data, description, param_grid, param_names, testset, test_data):
    ''' Return best HR value and best set of hyperparameters '''
    ...
    return best_hr, best_params

In [ ]:
best_hr, best_params = hp_selection(train_data, data_description, param_grid, param_names, testset, test_data)

In [ ]:
# Train the model with the best hyper-parameters and report HR and MRR on test dataset.
hr, mrr = ...
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')

# 5. Analysis (5 pts) 

Examine how the loss function, $||X||$ and $||Y||$ change during the training. Complete the function to obtain the history of losses values, $||X||$ and $||Y||$ for every epoch. Use best hyper-parameters for model config. Plot the graphs and discuss the results.

In [ ]:
def model_analysis(model_config, trainset, trainset_description):
    ...    
    return X_norms, Y_norms, losses

In [ ]:
X_norms, Y_norms, losses = model_analysis(best_params, train_data, data_description)

In [ ]:
# Plot the graphs and discuss the results.
...

What is the time complexity of the federated collaborative filterng with gradient descent? 